In [4]:
import pandas as pd
from scipy import spatial
import math

In [37]:
%%time
gnaf_address_combined = pd.read_csv("../data/Master/ACT-GNAF.csv")

<decorator-gen-55>:2: DtypeWarning: Columns (9,12,13) have mixed types.Specify dtype option on import or set low_memory=False.


Wall time: 3.84 s


In [38]:
%%time
asgs = pd.read_csv("../data/Master/" +"AU-ASGS.csv", dtype='str')

Wall time: 2.78 s


In [39]:
gnaf_address_combined.head(2)
gnaf_address_combined.shape

(255287, 44)

### Independent Testing

In [40]:
#### Method to find Cartesian Coordinates
def cartesian(latitude, longitude, elevation = 0):
    # Convert to radians
    latitude = latitude * (math.pi / 180)
    longitude = longitude * (math.pi / 180)

    R = 6371 # 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(latitude) * math.cos(longitude)
    Y = R * math.cos(latitude) * math.sin(longitude)
    Z = R * math.sin(latitude)
    return (X, Y, Z)

cartesian(*list((41.49008, -71.312796)))

(1529.0606767794266, -4520.739477097018, 4220.726125555107)

In [45]:
%%time

#### This could be done on master dataset creation ####

places = []

X = gnaf_address_combined[['LATITUDE', 'LONGITUDE']]

for index, row in X.iterrows():
    coordinates = [row['LATITUDE'], row['LONGITUDE']]
    cartesian_coord = cartesian(*coordinates)
    places.append(cartesian_coord)

Wall time: 25.5 s


In [54]:
Y = pd.DataFrame({"A": [1,3,5], "B": [5,6,7]})

values = []
for index, row in Y.iterrows():
#     coordinates = [row['A'], row['B']]
#     cartesian_coord = cartesian(*coordinates)
    values.append(row["A"])
print(values)

[1, 3, 5]


In [46]:
print(len(places))
print(places[4])

255287
(-4466.077465926709, 2672.6317151305593, -3674.347939923484)


In [55]:
%%time
##### Test with simulated Input Using Spatial KDTree to find the nearested coordinate in the dataset
##### This returns the index of the row in the dataset
from scipy import spatial

# lat = -35.19678041
# lon = 149.02779517

# lat = -35.220722
# lon = 149.102358

lon = 149.027961
lat = -35.243624

cartesian_coord = cartesian(lat, lon)

tree = spatial.KDTree(places)
closest = tree.query([cartesian_coord], p = 2)

index = closest[1][0]
print(index)

255286
Wall time: 12.2 s


In [56]:
### Get the full Address by the returned Index
gnaf_address_combined.iloc[index,:][['ADDRESS_DETAIL_PID', 'LONGITUDE', 'LATITUDE', 'FULL_ADDRESS']]

ADDRESS_DETAIL_PID                                   GAACT718835355
LONGITUDE                                                   149.028
LATITUDE                                                   -35.2436
FULL_ADDRESS          121 WILLIAM HOVELL DRIVE, BELCONNEN, ACT 2617
Name: 255286, dtype: object

### Make function of the Above experiment

In [6]:
AU = {}
AU['MESH_BLOCK'] = 'MB_CODE_2016'
AU['SA1'] = 'SA1_7DIGITCODE_2016'
AU['SA2'] = 'SA2_NAME_2016'
AU['SA3'] = 'SA3_NAME_2016'
AU['SA4'] = 'SA4_NAME_2016'
AU['SUBURB'] = 'SSC_NAME_2016'
AU['LGA'] = 'LGA_NAME_2016'
AU['STATE'] = 'STATE_NAME_2016'

country = AU

In [23]:
gnaf_address_combined[['CARTESIAN_COOR']]

,CARTESIAN_COOR
0,"(-4466.081815343984, 2672.6451730840595, -3674..."
1,"(-4466.081815343984, 2672.6451730840595, -3674..."
2,"(-4466.071446037419, 2672.6110625341303, -3674..."
3,"(-4466.081815343984, 2672.6451730840595, -3674..."
4,"(-4466.077465926709, 2672.6317151305593, -3674..."
...,...
255282,"(-4459.481882179927, 2675.3416170449, -3680.38..."
255283,"(-4461.520524934195, 2678.6310419069455, -3675..."
255284,"(-4461.177948854766, 2677.07941906673, -3677.0..."
255285,"(-4461.874958606976, 2674.5440704526345, -3678..."


In [57]:
def getCartesianCoordinatesArray():
    cart_cor = []
    for index, row in gnaf_address_combined[['CARTESIAN_COOR']].iterrows():
        cart_cor.append(row['CARTESIAN_COOR'])
    return cart_cor

In [58]:
def getRegionByCoordinates(lat, lon):
    
    cartesian_coord = cartesian(lat, lon)
    
    places = getCartesianCoordinatesArray()

    tree = spatial.KDTree(places)
    closest = tree.query([cartesian_coord], p = 2)
    
    index = closest[1][0]
    
    ## Get the result row
    res = gnaf_address_combined.iloc[index,:]
    
    
    mb = res['MB_2016_PID']
    
    region = asgs[asgs['MB_CODE_2016'] == mb[4:]]

    for key, value in AU.items():
        print(key+":"+region.iloc[0][value])

In [59]:
%%time
lon = 149.027961
lat = -35.243624

getRegionByCoordinates(lat, lon)

ValueError: not enough values to unpack (expected 2, got 1)